### Create a notebook to perform Runge-Kutta for multiple coupled variables

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

### Define our coupled derivatives to integrate

In [ ]:
def dydx(x,y):
    #set the derivatives
    #our eq is d^2y/dx^2 = -y
    #so write
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0]
    #we will set z = y[1]
    
    #declare an array, size 2
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -1
    y_derivs[1] = -1*y[0]
    
    #return an array
    return y_derivs

### Def. the 4th order RK method

In [ ]:
def rk4_mv_core(dydx, xi, yi, nv, h):
    
    #declare k? arrays
    #h = step size
    #yi = array
    #nv = number of variables

    #each new k depends on the previous k's, K4 depends on k3 to k1
    #nv in our case is 2, y and z
    
    k1 =  np.zeros(nv)
    k2 =  np.zeros(nv)
    k3 =  np.zeros(nv)
    k4 =  np.zeros(nv)
    
    #define x at 1/2 step
    x_ipoh = xi +0.5*h
    
    #define x at 1 step
    x_ipo = xi + h
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + 0.5*k3[:]
    y_derivs = dydx(x_ipo, y_temp)
    k4[:] = h*y_derivs[:]
    
    #advance y by a step h
    #weighted sum
    yipo = yi + (k1 + 2*k2+ 2*k3 +k4)/6.
    
    return yipo

### Def an adaptive step size driver for RK4


In [ ]:
def rk4_mv_ad(dydx, x_i, y_i, nv, h, tol):
    
    #define safety scale
    #used to estimate an error, 1 big step, 2 little steps
    #if error is bigger than tol, reduce size of step
    #if error is smaller than tol, increase size of step, limit in 
    #increase of step size, half of previous step
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    
    i = 0
    
    #create an error
    Delta = np.full(nv, 2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        
            #estimate our error by taking one step of size h 
            #vs. two steps of size h/2
            y_2 = rk4_mv_core(dydx, x_i, y_i, nv, h_step)
            y_1 = rk4_mv_core(dydx, x_i, y_i, nv, 0.5*h_step)
            y_11 = rk4_mv_core(dydx, x_i+0.5*h_step, y_i, nv, 0.5*h_step)
            
            #compute error
            Delta = np.fabs(y_2 - y_11)
            
            #if the error is too lage, take smaller step
            if(Delta.max()/tol > 1.0):
                #our error is too large, decrease the step
                h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
                
            #check iteration
            
            if(i>imax):
                print("Too many iterations in rk4_mv_ad()")
                raise StopIteration("Ending after i = ", i)
                
            #iterate 
            i += 1
            
    #next time, try to take a bigger step
    #can only take a step thats 2x as big as current step, due to h_new_fac
    h_new = np. fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step
                
            

In [ ]:
def rk4_mv(dfdx, a, b, y_a, tol):
    #dfdx is the derivative wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerane for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 *(b-a)
    
    #set a max number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set the number of coupled odes to the 
    #size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv), y_a)
    
    #set a flag
    flag = 1